Retrieve Data

In [4]:
import tensorflow as tf
import os
base_dir="/kaggle"
input_dir = os.path.join(base_dir, "input")
project_dir = os.path.join(input_dir, "binary-adjusted-pe")
dataset_dir = os.path.join(project_dir, "Binary_Adjusted")
batch_size=128
dataset = tf.keras.utils.image_dataset_from_directory(
  directory=dataset_dir,
  validation_split=0.2,
  subset="both",
  seed=123,
  image_size=(512, 512),
  color_mode="rgb",
  label_mode="binary",
  batch_size=batch_size)
AUTOTUNE=tf.data.AUTOTUNE
training_data = dataset[0].prefetch(buffer_size=AUTOTUNE)
valid_data= dataset[1].prefetch(buffer_size=AUTOTUNE)

D0502 20:45:39.296361623    1592 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0502 20:45:39.296393328    1592 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0502 20:45:39.296397325    1592 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0502 20:45:39.296400212    1592 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0502 20:45:39.296403149    1592 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0502 20:45:39.296405912    1592 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0502 20:45:39.296408375    1592 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0502 20:45:39.

Found 174445 files belonging to 2 classes.
Using 139556 files for training.
Using 34889 files for validation.


In [6]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

In [ ]:
### SETTING UP THE BASE MODEL

import keras
import tensorflow as tf
# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    base_model = tf.keras.applications.VGG19(input_shape = (512, 512, 3), include_top = False, weights = 'imagenet')
    base_model.trainable= False
    inputs = keras.Input(shape=(512, 512, 3))
    x=base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    #x = tf.keras.layers.Dropout(0.2)(x)
    layerinfo="base model to output no dropout"
    x = tf.keras.layers.Dense(1024,activation="relu")(x)
    outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)



### MODEL FITTING #######
    lr=0.0005
    optimizer="Adam"
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=[
                  tf.keras.metrics.AUC(),
				  tf.keras.metrics.BinaryAccuracy(),
				  tf.keras.metrics.Precision(),
				  tf.keras.metrics.Recall(),
				  tf.keras.metrics.TruePositives(),
				  tf.keras.metrics.TrueNegatives(),
				  tf.keras.metrics.FalsePositives(),
				  tf.keras.metrics.FalseNegatives()])
    model.summary()

In [28]:
### Continue Training #######
base_model = tf.keras.applications.VGG19(input_shape = (512, 512, 3), include_top = False, weights = 'imagenet')
base_model.trainable=True
print("Number of layers in the base model: ", len(base_model.layers))
fine_tune_at = (len(base_model.layers)-10)
with tpu_strategy.scope():
    models_path=r"/kaggle/input/the-champion/VGG19-1024--ep-0019.h5"
    model = tf.keras.models.load_model(models_path, compile=False)
    lr=0.00025
    model.trainable=True
    for layer in model.layers[:fine_tune_at]:
        layer.trainable = False
        print(layer)
    optimizer="Adam"
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=[
                  tf.keras.metrics.AUC(),
				  tf.keras.metrics.BinaryAccuracy(),
				  tf.keras.metrics.Precision(),
				  tf.keras.metrics.Recall(),
				  tf.keras.metrics.TruePositives(),
				  tf.keras.metrics.TrueNegatives(),
				  tf.keras.metrics.FalsePositives(),
				  tf.keras.metrics.FalseNegatives()])
model.summary()

Number of layers in the base model:  22
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 16, 16, 512)       20024384  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 20,550,721
Trainable params: 0
Non-trainable params: 20,550,721
_______

In [ ]:
model.evaluate(valid_data, verbose=1)

In [23]:
ep=20
model.summary()
history = model.fit(training_data,validation_data=valid_data,epochs=ep,callbacks=[save_every_epoch], verbose=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 16, 16, 512)       20024384  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 20,550,721
Trainable params: 0
Non-trainable params: 20,550,721
_______________________________________________

2023-05-02 21:31:46.609126: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-05-02 21:31:46.763016: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


KeyboardInterrupt: 

In [9]:
import os

working_dir=os.path.join(base_dir,"working")
MODEL_NAME="VGGnm19"
MODEL_SERIES="-1024-19FineTuning"
MODELS_PATH=os.path.join(working_dir,MODEL_NAME+MODEL_SERIES+"-ep-{epoch:04d}.h5")
working_dir
save_every_epoch=tf.keras.callbacks.ModelCheckpoint(
    MODELS_PATH,
    save_freq="epoch",
    verbose=1
)
MODELS_PATH

'/kaggle/working/VGG19-1024-19FineTuning-ep-{epoch:04d}.h5'